In [ ]:
import os
import pandas as pd
import numpy as np
import csv
from datetime import datetime

# Using Concise Logic Statements


In [ ]:
# import Apoorva's CSV
nc_df = pd.read_csv("Data/NC_Python_results.csv")

# Verify dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print(nc_df.columns)
print(nc_df.iloc[:25])

Index(['id', 'year_birth', 'zipcode', 'screening_created_at', 'scope',
       'county', 'state', 'court_jurisdiction', 'c_id', 'charge',
       'charge_type', 'disposition', 'disposition_date', 'sentence',
       'displayed_charge_yn', 'incidentidentifier',
       'Disposition_Normalization', 'ChargeType_Normalization',
       'flag_disposition', 'sent', 'Keywords', 'iscodable', 'min,max', 'fine',
       'fee', 'Restitution', 'cost', 'Balance', 'Jail', 'Prison', 'Credit',
       'probation', 'parole', 'Community_hrs', 'Diversion', 'soft_time',
       'hard_time', 'net_hard_time', 'Net_hard_time', 'Sentence_time',
       'sentence_time'],
      dtype='object')
                          id  year_birth  zipcode screening_created_at  scope  \
0   c53a005b0ba339fdd1870787        1980  27858.0            3/21/2017    7.0   
1   c53a005b0ba339fdd1870787        1980  27858.0            3/21/2017    7.0   
2   c53a005b0ba339fdd1870787        1980  27858.0            3/21/2017    7.0   
3   c53a

In [ ]:
print(nc_df["ChargeType_Normalization"].value_counts())
none_df = nc_df[nc_df["ChargeType_Normalization"].str.contains("adfsfasfdas",na=True)]
print(len(none_df.index))
print(3371/(11764 + 11425 + 5012 + 3371))

minor             11764
misdemeanor       11425
felony             5012
not classified     3371
Name: ChargeType_Normalization, dtype: int64
0
0.1067718231344229


In [ ]:
val = 0
for index, row in nc_df.iterrows():
    if pd.isna(row["Disposition_Normalization"]):
        val += 1
print(val)
print(index)

print(nc_df["Disposition_Normalization"].value_counts())
none_df = nc_df[nc_df["Disposition_Normalization"].str.contains("adfsfasfdas",na=True)]
print(len(none_df.index))



0
31571
not_guilty/dismissed    14708
guilty                   9534
likely_pending-blank     3309
unknown                  3145
pending                   876
Name: Disposition_Normalization, dtype: int64
0


In [ ]:
print((3145)/(14708 + 9534 + 3309 + 3145 + 876))

0.09961358165463069


In [ ]:
print(31571 * 0.9)
a = nc_df["charge"].value_counts()
val = 0
counter = 0
for index, value in a.items():
    counter += 1
    val += value
    if val > 31571 * 0.90:
        print(index)
        print(counter)
        break

28413.9
MOTION FOR FTA IN ERROR (STATUTE: 15A-951)
405


In [ ]:
# Convictions logic
#

# Nonconvictions logic
#   Check for felony convictions
#   If no felony convictions, any conviction with [Disposition_normlization] found "not guilty" or "dismissed" is eligible
print(nc_df.charge.value_counts())
nc_df.charge.value_counts().to_csv("nc_charge_counts.csv")
nc_df.charge

UNAVAILABLE                                                                                          3268
EXPIRED REGISTRATION CARD/TAG (STATUTE: 20-111(2))                                                   1438
DWLR (STATUTE: 20-28(A))                                                                             1107
NO OPERATORS LICENSE (STATUTE: 20-7(A))                                                              1021
SPEEDING (STATUTE: 20-141(B))                                                                         906
SIMPLE WORTHLESS CHECK (STATUTE: 14-107(D)(1))                                                        722
EXPIRED/NO INSPECTION (STATUTE: 20-183.8(A)(1))                                                       704
OPERATE VEH NO INS (STATUTE: 20-313(A))                                                               692
IMPROPER EQUIP - SPEEDOMETER (STATUTE: 20-123.2)                                                      644
SPEEDING (STATUTE: 20-141(J1))                

0                           Att Obtain Prop False Pretense
1                                          Insurance Fraud
2                           False Report To Police Station
3                           Att Obtain Prop False Pretense
4                                          Insurance Fraud
5                           False Report To Police Station
6                            Expired Registration Card/Tag
7                               Possess Drug Paraphernalia
8                             Misd Prob Viol Out Of County
9                             Misd Prob Viol Out Of County
10                                       Felony Conspiracy
11                                     Assault On A Female
12                           Robbery With Dangerous Weapon
13                            Assault With A Deadly Weapon
14                                     Assault On A Female
15                              Possess Drug Paraphernalia
16                              Misdemeanor Probation Vi

In [ ]:
# check for pending charges. If anyone has pending charges, remember their id, to disqualify them from conviction eligibility
# calculate completion time for each charge in the dataset (disposition_date + sentence time), measure if they pass 5/10 year limits
# check for felony convictions. If anyone has a felony conviction, remember their id

# for each charge in the dataset:
#.  if charge is a conviction ("guilty" in disposition_normalized)
#     if it is a misdemeanor ("misdemeanor" in ChargeType_Normalization):
#         check if it is not an A1 misdemeanor (using Joe's brute force sheet)
#         if it is not, check the time since completion time
#         if more than 5 years between 06/01/2020 and sentence_completion (calculated previously), eligible

#     if it is a felony:
#         check if it is not an A-G felony (using Joe's brute force sheet)
#.        if not, check time since completion
#.        if more than 10 years between 06/01/2020 and sentence_completion (calculated previously), eligible
#.
#    if charge is a nonconviction ("not convicted/dismissed" in disposition_normalized):
#       check if person has a felony conviction. If not, eligible
#.
#.   All other charges considered not eligible
#.
#    When counting, if person has multiple felonies with different disposition dates, they cannot be considered "fully eligible"

In [ ]:
# make brute force dictionary
colnames = ['Offense',"Eligibility"]
brute_force = pd.read_csv("brute_force.csv",names=colnames)
print(brute_force.nunique())
#print(brute_force)
#brute_force_dict = brute_force.to_dict()
brute_force_dict = {}
count = 0
with open('brute_force.csv') as file:
    csv_reader = csv.reader(file, delimiter=';')
    for row in csv_reader:
        count += 1
        print(count)
        print(row)
        brute_force_dict[row[0]] = row[1].lower()
print(brute_force_dict)
print(len(brute_force_dict))

Offense        215
Eligibility      4
dtype: int64
1
['Prose description', 'Eligibilty']
2
['EXPIRED REGISTRATION CARD/TAG (STATUTE: 20-111(2))', 'Yes']
3
['DWLR (STATUTE: 20-28(A))', 'Yes']
4
['NO OPERATORS LICENSE (STATUTE: 20-7(A))', 'Yes ']
5
['SPEEDING (STATUTE: 20-141(B))', 'Yes']
6
['SIMPLE WORTHLESS CHECK (STATUTE: 14-107(D)(1))', 'Yes']
7
['EXPIRED/NO INSPECTION (STATUTE: 20-183.8(A)(1))', 'Yes']
8
['OPERATE VEH NO INS (STATUTE: 20-313(A))', 'Yes']
9
['IMPROPER EQUIP - SPEEDOMETER (STATUTE: 20-123.2)', 'Yes']
10
['SPEEDING (STATUTE: 20-141(J1))', 'Yes']
11
['MISDEMEANOR LARCENY (STATUTE: 14-72(A))', 'Yes']
12
['OBTAIN PROPERTY FALSE PRETENSE (STATUTE: 14-100)', 'No']
13
['POSSESS MARIJUANA UP TO 1/2 OZ (STATUTE: 90-95(D)(4))', 'Yes ']
14
['POSSESS DRUG PARAPHERNALIA (STATUTE: 90-113.22(A))', 'Yes']
15
['MISDEMEANOR PROBATION VIOL (STATUTE: 15A-1345)', 'Not sure']
16
['RESISTING PUBLIC OFFICER (STATUTE: 14-223)', 'Yes']
17
['ASSAULT ON A FEMALE (STATUTE: 14-33(C)(2))', 'No']
18

In [ ]:
# make dict of charges for concurrency test
ids = nc_df["id"].unique()
final_dict = {}
for id in ids:
    #print(id)
    final_dict[str(id)] = []
    sub_df = nc_df[nc_df["id"] == id]
    conviction_sub_df = sub_df[sub_df["Disposition_Normalization"].str.match("guilty",na=False)]
    times = conviction_sub_df["disposition_date"]
    for time in times:
        print(time)
        new_time = pd.to_datetime(time, infer_datetime_format=True)
        final_dict[str(id)].append(new_time)

7/9/2014
7/9/2014
8/21/2012
7/9/2014
7/9/2014
7/9/2014
7/9/2014
7/9/2014
7/9/2014
7/9/2014
7/9/2014
1/28/2002
10/28/2002
1/5/2004
7/14/2008
8/21/2012
1/28/2002
10/28/2002
1/5/2004
7/14/2008
8/21/2012
8/21/2012
8/21/2012
8/21/2012
8/21/2012
7/26/2011
7/26/2011
6/22/2009
1/18/2011
1/18/2011
1/18/2011
6/5/2013
1/18/2011
12/14/2012
12/14/2012
12/14/2012
12/8/2010
11/22/2011
7/15/2013
6/28/2013
1/24/2014
1/12/2018
1/12/2018
1/12/2018
3/7/2014
3/7/2014
3/7/2014
3/7/2014
3/7/2014
9/14/2000
9/14/2000
9/14/2000
9/14/2000
10/12/2005
10/12/2005
8/14/2007
9/14/2000
9/14/2000
9/14/2000
9/14/2000
4/25/2006
10/12/2005
10/12/2005
8/14/2007
4/17/2006
4/17/2006
3/24/2011
5/16/2011
10/15/2014
2/6/2014
2/6/2014
11/20/2001
4/16/2009
7/8/2013
12/2/1999
12/2/1999
12/2/1999
12/2/1999
12/2/1999
11/20/2001
11/20/2001
11/20/2001
11/20/2001
11/20/2001
11/20/2001
11/20/2001
11/20/2001
1/9/2001
4/16/2009
7/8/2013
11/2/1998
11/2/1998
11/2/1998
5/10/2013
10/7/1998
4/27/1999
10/25/2000
1/14/2008
10/25/2000
3/15/2006
1

3/12/1992
3/12/1992
10/22/1992
10/22/1992
10/22/1992
4/9/2015
12/5/2000
5/13/2002
4/2/2015
7/25/2000
7/25/2000
7/25/2000
7/25/2000
7/25/2000
7/25/2000
7/25/2000
7/25/2000
7/25/2000
12/1/2000
7/25/2000
7/25/2000
7/25/2000
7/25/2000
7/25/2000
7/25/2000
7/25/2000
6/30/1999
7/25/2000
7/25/2000
12/1/2000
7/25/2000
7/25/2000
7/25/2000
7/25/2000
7/25/2000
6/30/1999
7/25/2000
7/25/2000
7/25/2000
7/25/2000
7/25/2000
7/25/2000
6/5/2000
8/3/2005
6/5/2000
1/13/1997
6/5/2000
5/1/2008
5/1/2008
10/17/2008
10/17/2008
10/30/2008
9/24/2009
9/7/2009
11/16/2010
5/13/2010
5/13/2010
5/13/2010
5/13/2010
5/13/2010
5/13/2010
11/16/2010
10/3/2007
10/3/2007
2/15/2011
2/15/2011
8/31/2015
10/3/2007
10/3/2007
2/15/2011
2/15/2011
8/31/2015
2/15/2011
2/15/2011
8/31/2015
2/15/2011
2/15/2011
8/31/2015
2/15/2011
2/15/2011
8/31/2015
3/9/2015
9/9/2016
3/9/2015
3/9/2015
3/9/2015
5/1/2008
5/1/2008
9/24/2009
9/24/2009
11/16/2010
5/13/2010
5/13/2010
5/13/2010
5/13/2010
11/16/2010
4/18/2013
6/4/2014
6/4/2014
11/21/2016
5/1/200

10/8/1999
9/12/2007
11/14/1986
11/28/1988
9/2/1993
7/24/1995
3/6/2006
3/9/2009
4/26/2012
4/26/2012
4/23/2013
3/9/2009
4/26/2012
4/26/2012
4/23/2013
12/6/2011
8/1/2013
5/23/1997
5/23/1997
5/23/1997
5/23/1997
5/23/1997
5/23/1997
10/31/2012
10/31/2012
7/14/2003
7/14/2003
1/31/2015
8/19/2008
8/19/2008
12/8/2010
12/8/2010
5/24/2011
10/10/2012
10/10/2012
6/13/2013
7/31/2014
1/14/2016
1/14/2016
1/14/2016
1/21/2016
3/1/2002
5/10/2005
5/26/2005
8/22/2006
11/8/2007
11/8/2007
8/19/2008
8/19/2008
12/8/2010
12/8/2010
5/24/2011
10/10/2012
10/10/2012
6/13/2013
7/31/2014
1/14/2016
1/14/2016
1/14/2016
11/30/2010
11/30/2010
11/30/2010
11/30/2010
8/13/2002
7/24/2007
9/18/2008
10/9/2015
11/26/2008
11/26/2008
5/10/1995
3/6/1996
8/7/1996
2/19/2014
2/19/2014
2/16/2011
2/16/2011
7/9/2015
2/5/2010
7/9/2015
6/26/1989
6/26/1989
12/15/1989
5/13/1996
5/13/1996
6/24/1996
3/23/1998
9/4/1990
4/3/1996
4/3/1996
10/8/1996
5/6/1996
7/5/1996
7/23/1996
7/23/1996
8/5/1996
10/8/1996
11/18/1997
9/24/1996
10/5/1993
5/10/1994
6

11/28/1989
2/28/1989
9/25/2017
7/18/1996
7/18/1996
7/18/1996
7/18/1996
7/18/1996
7/18/1996
7/18/1996
7/18/1996
6/27/1996
7/18/1996
7/17/1996
7/18/1996
7/18/1996
7/18/1996
7/18/1996
1/14/2010
12/7/1995
2/17/2009
3/9/2009
2/25/1986
8/11/1992
3/8/1994
3/8/1994
6/10/1996
5/28/1996
8/21/1996
3/21/2000
2/4/2002
2/8/2002
5/20/2014
1/5/2016
3/26/2012
5/20/2014
1/5/2016
6/3/1992
4/18/1989
9/3/1991
3/29/2004
3/29/2004
7/15/1998
5/18/2017
5/18/2017
7/6/2016
6/8/2007
6/8/2007
3/8/1988
1/20/1988
1/20/1988
10/23/1997
7/27/1983
3/15/1985
1/21/1986
9/30/1985
2/18/1988
2/18/1988
1/6/1988
12/27/1989
1/6/1988
8/1/2006
8/1/2006
7/31/1990
9/14/1990
9/14/1990
9/14/1990
6/2/2004
6/2/2004
6/2/2004
6/2/2004
6/2/2004
6/2/2004
6/2/2004
6/2/2004
6/2/2004
6/2/2004
6/2/2004
6/2/2004
6/2/2004
6/2/2004
6/2/2004
6/2/2004
6/2/2004
6/2/2004
6/2/2004
6/2/2004
6/2/2004
6/2/2004
6/2/2004
6/2/2004
6/2/2004
6/2/2004
6/2/2004
6/2/2004
6/2/2004
6/2/2004
2/7/1992
2/7/1992
2/7/1992
2/7/1992
2/7/1992
2/7/1992
3/24/2000
1/30/2017


12/4/2007
12/4/2007
12/4/2007
12/4/2007
9/11/2009
9/11/2009
9/11/2009
5/16/2013
12/4/2007
9/11/2009
12/4/2007
9/11/2009
2/5/2007
2/5/2007
2/5/2007
4/23/2009
3/12/2009
3/12/2009
6/9/2010
3/12/2009
3/12/2009
1/8/2013
1/8/2013
12/4/2009
8/17/2017
10/12/2016
3/14/2013
7/10/2001
8/19/2002
1/11/2005
3/14/2013
12/3/1985
6/2/1986
10/19/1987
8/26/1988
6/20/1989
4/13/1993
4/13/1993
10/19/1993
10/19/1993
10/19/1993
12/14/1983
12/3/1984
12/3/1984
9/11/1985
9/11/1985
3/4/1987
1/17/1996
6/3/1998
6/23/1999
6/3/2003
8/18/1989
10/2/1991
3/11/1986
1/14/2015
1/14/2015
1/14/2015
1/14/2015
4/21/2015
7/23/2018
12/20/2002
12/20/2002
1/23/2004
10/19/2011
5/2/2005
8/1/2008
8/1/2008
5/2/2005
2/27/2006
4/23/2009
1/14/2010
4/23/2009
4/23/2009
1/14/2010
3/27/2009
11/20/2013
11/5/2013
3/13/2007
8/6/2007
10/30/2007
11/20/2013
11/5/2013
8/7/2015
3/13/2017
7/28/2017
11/6/2014
11/6/2014
9/17/2009
11/8/2013
11/8/2013
8/5/2009
9/17/2009
8/25/2009
8/5/2009
9/17/2009
8/25/2009
8/19/2002
8/19/2002
12/19/2007
9/7/2006
11/27/

...

3/1/2011
11/27/2017
5/12/2004
5/12/2004
5/12/2004
5/12/2004
5/12/2004
5/12/2004
5/12/2004
5/12/2004
8/4/2004
4/12/2004
12/8/2005
7/14/2005
7/8/2008
7/8/2008
6/25/2007
7/3/2008
1/25/2018
2/13/2015
9/6/2017
6/4/2008
4/13/2006
11/18/2003
11/18/2003
2/20/2014
2/20/2014
1/20/1995
1/20/1995
10/19/1995
11/30/2000
10/24/2007
7/24/2000
5/25/2018
5/10/2016
9/22/2015
3/3/2000
3/8/2010
3/3/2000
3/8/2010
4/2/2009
5/5/2014
4/2/2009
10/1/2013
5/5/2014
1/20/1988
1/20/1988
10/28/1996
8/30/1999
5/4/2001
5/4/2001
3/6/2017
11/21/2013
2/8/2017
10/21/2013
10/21/2013
2/22/2012
2/27/2014
9/23/2002
9/23/2002
2/19/2003
11/16/2010
10/14/2010
3/13/2013
8/14/2006
10/23/2007
8/9/2013
7/2/2015
12/5/2017
8/13/2013
3/10/2014
1/7/2002
9/15/2003
2/25/2003
2/16/2004
9/14/2004
1/12/2005
1/12/2005
5/5/2005
4/30/2007
4/30/2007
4/30/2007
4/30/2007
8/15/2016
2/19/2016
5/31/2006
2/8/2012
2/15/1999
2/25/2004
9/15/2009
12/12/1997
5/14/2015
4/8/2016
3/25/2009
7/26/2012
8/1/2012
3/9/2012
7/18/2012
4/17/2007
12/15/1994
2/22/1996
2/

9/3/2008
7/22/1987
7/27/1987
5/1/1989
9/1/1988
12/14/1988
7/10/1989
1/26/1989
1/26/1989
3/17/1989
3/3/1989
3/23/1990
3/23/1990
3/12/2004
3/12/2004
3/12/2004
3/12/2004
1/28/2004
7/2/2003
3/10/2004
3/10/2004
12/29/2014
2/9/2015
1/28/2004
7/2/2003
3/10/2004
3/10/2004
12/29/2014
3/12/1998
3/12/1998
3/12/1998
5/18/1998
5/18/1998
5/18/1998
5/18/1998
1/28/2003
11/17/2005
11/17/2005
1/6/2006
3/14/2006
8/25/2006
11/6/2008
11/6/2008
1/28/2003
11/17/2005
11/17/2005
1/6/2006
3/14/2006
8/25/2006
11/6/2008
11/6/2008
2/29/1996
11/21/1996
5/19/2008
1/13/2009
4/1/2010
7/18/2017
12/12/2016
12/12/2016
12/28/2000
2/16/2001
2/16/2001
9/12/2001
7/8/2003
7/8/2003
12/5/2007
8/17/2009
8/17/2009
11/13/2009
12/28/2000
2/16/2001
2/16/2001
9/12/2001
7/8/2003
7/8/2003
12/5/2007
8/17/2009
8/17/2009
11/13/2009
9/6/2016
10/12/2017
10/12/2017
3/24/2016
6/4/2006
3/9/2006
2/1/2012
2/1/2012
2/1/2012
2/1/2012
4/13/1993
4/13/1993
4/13/1993
4/13/1993
7/7/1994
3/6/1995
5/24/1993
7/12/1993
7/12/1993
7/12/1993
7/12/1993
7/12/19

2/24/2009
3/16/2011
4/27/2011
9/9/2011
8/9/2017
11/8/1996
11/17/2003
8/11/1992
4/15/1993
1/12/2001
7/28/2006
3/2/2018
8/30/1994
10/6/2008
10/6/2008
2/5/2009
2/5/2009
2/12/2002
11/18/2002
11/18/2002
11/18/2002
5/26/2011
3/4/2002
12/5/2017
10/16/2007
5/3/2011
1/28/2013
1/28/2013
1/28/2013
11/13/2015
11/13/2015
11/13/2015
3/29/2005
8/25/1997
8/25/1997
9/28/1994
9/28/1994
10/4/2002
1/24/2006
1/22/1996
1/22/1996
7/21/1998
8/20/1999
6/26/2017
3/20/2018
12/13/2011
1/13/2011
2/29/2012
2/29/2012
7/24/2012
10/14/2010
6/25/1999
6/25/1999
4/22/1999
9/19/2016
9/21/2017
5/25/2010
1/4/2016
5/13/2014
7/27/1989
2/17/1993
1/11/2012
7/30/1984
7/30/1984
7/30/1984
7/30/1984
8/18/1986
8/18/1986
11/19/1987
11/25/1987
7/11/1989
5/8/1990
2/8/1990
6/16/1994
9/10/1996
11/21/1996
6/5/2009
8/30/2006
8/30/2006
1/22/2009
1/22/2009
12/1/2008
1/6/2010
8/16/2010
12/1/2008
1/6/2010
8/16/2010
11/15/2011
5/21/2014
4/25/1986
4/7/1987
2/26/1987
1/5/1988
5/21/2014
8/11/2017
1/10/1996
1/10/1996
1/10/2005
3/8/2010
6/14/2004
6/

6/13/2005
4/26/2016
3/31/2015
12/18/2015
8/28/2009
9/15/2016
9/15/2016
9/15/2016
9/4/2015
12/3/2007
3/11/2008
12/16/1996
8/19/2013
8/14/2007
8/14/2007
8/15/2011
8/15/2011
4/16/2012
4/16/2012
3/25/2002
3/25/2002
11/25/1996
7/19/2004
1/11/2002
4/29/1986
3/13/1987
1/7/2009
1/7/2009
10/2/2009
10/2/2009
1/5/1989
1/5/1989
9/28/1994
8/20/1991
8/20/1991
9/28/1994
9/15/1995
9/15/1995
10/7/1996
9/30/1996
8/1/2008
2/24/1997
2/28/1997
8/13/2001
8/13/2001
4/22/1999
4/22/1999
4/22/1999
11/5/2013
1/27/2014
4/16/2010
4/16/2010
6/9/2005
2/14/1992
11/2/1994
11/2/1994
5/12/1995
4/9/1996
11/20/1995
11/20/1995
2/8/1996
4/9/1996
8/22/1996
8/22/1996
2/9/2011
2/9/2011
9/24/2002
6/22/2004
6/22/2004
6/22/2004
6/22/2004
8/1/1997
5/19/1998
5/19/1998
8/9/1999
8/7/2000
12/1/1999
12/1/1999
12/1/1999
12/1/1999
5/7/2014
5/7/2014
9/24/2002
4/28/2006
7/2/1996
9/26/1995
9/26/1995
9/26/1995
9/26/1995
9/26/1995
9/26/1995
9/26/1995
9/26/1995
9/26/1995
9/26/1995
12/5/1995
7/30/1997
9/11/1997
9/11/1997
8/7/1997
9/11/1997
4/23

2/17/2009
11/4/2008
11/4/2008
2/20/2017
2/24/2005
9/18/2013
4/30/2014
9/20/2012
9/20/2012
9/20/2012
9/20/2012
12/13/2004
12/13/2004
2/9/2006
7/20/2009
6/14/2017
5/29/2003
5/29/2003
6/22/2004
6/22/2004
8/9/2017
3/8/2005
9/28/1992
11/18/1992
10/19/2015
10/19/2015
2/22/2011
2/22/2011
2/22/2011
2/22/2011
10/7/2008
7/9/1993
8/8/1994
8/8/1994
12/21/1994
12/8/1998
6/13/1989
9/10/1996
8/8/1989
1/24/2008
5/21/2001
5/21/2001
5/21/2001
9/8/1999
9/8/1999
9/8/1999
1/16/2007
2/28/1983
3/31/1983
11/7/1983
4/30/1986
5/23/2006
5/23/2006
7/31/1991
7/31/1991
7/31/1991
9/8/1992
7/28/1982
5/6/2003
10/26/2006
2/12/1988
2/12/1988
8/15/2008
12/8/2011
2/6/1996
6/6/2005
6/6/2005
6/6/2005
7/12/2012
6/3/1999
5/25/1988
5/25/1988
5/25/1988
5/25/1988
8/7/1985
7/20/1989
12/14/1994
1/18/2006
4/19/1993
12/11/2015
3/9/1995
12/11/2009
4/27/2011
5/27/2011
6/23/2014
7/27/1994
6/21/1999
9/20/2011
1/23/2015
1/14/1983
9/3/2002
4/20/2004
5/13/2016
6/22/1999
7/15/2016
5/5/2010
5/5/2010
5/5/2010
5/5/2010
3/13/2008
10/6/2008
9/11

In [ ]:
nc_df["disposition_date"] = pd.to_datetime(nc_df["disposition_date"], infer_datetime_format=True)
print(nc_df.disposition_date.min())
print(nc_df.disposition_date.max())

1976-05-26 00:00:00
2018-10-25 00:00:00


In [ ]:
# check for pending charges, get list of ids

pending_df = nc_df[nc_df["Disposition_Normalization"].str.contains("pending",na=False)]
print(pending_df.id.nunique())
pending_ids = pending_df.id.unique()
print(pending_ids)

guilty_df = nc_df[nc_df["Disposition_Normalization"].str.contains("guilty",na=False)]
print(guilty_df.id.nunique())
guilty_felony_df = guilty_df[guilty_df["ChargeType_Normalization"].str.contains("felony",na=False)]
guilty_felony_ids = guilty_felony_df.id.unique()
print(guilty_felony_ids)

354
['f38b52f8a17d908dc50dab90' 'd715eda819aeabff926335ac'
 '51e99fbfd63a7dc09a9db6e4' '0a3ae631d436d56b0041c34d'
 'f438453a387597c1746d4587' '10295d063920b84cc1f32e31'
 '67ca34ab7767859a1bcb7adc' 'eac76249f89f5e63ceb9a9a1'
 'e607c5090c82cf562d6b9f4e' 'e88b9f7e48314f42e3726407'
 '7a3ccd36eeabc9269c8b2d4f' '17c984e220b0da93b7e67bf1'
 'bf6bec94cd10d1a49322c980' '21468910a8cff3c34fe44cc3'
 '92072bca7a2844608a2fe90f' '032de70f13dc94bad383676f'
 '4bb5c7b1aadacda9266a46a3' '5c529b033930a89167d8775c'
 'd02db0c1e8492f86497907f1' '67b540016930b47a5a6b4630'
 '5fb37603a329bd91759bce3f' '11fde951e85d22a42912ae65'
 'f5e0b23ce35b578b3ae81c65' '47e32eae331f11491739c649'
 'bf3259ea58bfb789d5052203' 'f3e07d3fd7d61d22690d5dc7'
 '13fab8f1d08ca026131a8ce0' 'bd4a36961901322f777bb494'
 '7489d268ea5bcd7bf89b5a5a' '8cf24e9dc6f9663a0c38ace9'
 '5966659204f29c8c628c9427' '5cbfb0daa242de38f2ceef43'
 'cea671b652c8ba6d3b1f71e4' '576fea3e689a6e539488ba9a'
 'a0578d8a832219ddf612a3a2' '9223de93b19e2845d9af009c'
 '11d5

['c53a005b0ba339fdd1870787' 'eb7ef75d5a2c273f67ed6c9e'
 '9d5e7512b7d2a03533c7a913' '2f31b3b2138370a66cfb12b8'
 'cf0b665ec8abd7784f124e37' 'baf0d3d116d68d57b571a49d'
 'a8ff0b0688b62423d412b2c3' 'e88b9f7e48314f42e3726407'
 '12b5b44f06b73414ee2ff57d' '4c830ae12fab749b0c3fb2d1'
 '67ca34ab7767859a1bcb7adc' '615a2d249845eea47f96a2e4'
 '2e9a5eca74eca57c1afe244b' '0b912cc5de49f05219a6ad2e'
 '7058979874829f5a48d86077' '32820d829f1528f0dc2fdd46'
 '31bf9a97c4508ecd0420be51' 'cd47fe6c858bf9bc56f1299d'
 '92072bca7a2844608a2fe90f' '6a77bff77d8a5a7af19ae07f'
 'f778ae5c4a7592f775e27e5b' '5c529b033930a89167d8775c'
 '5fb37603a329bd91759bce3f' '13fab8f1d08ca026131a8ce0'
 'bd4a36961901322f777bb494' '7489d268ea5bcd7bf89b5a5a'
 '5cbfb0daa242de38f2ceef43' 'a0578d8a832219ddf612a3a2'
 '49e3f7c94fc6db5f6c7101e4' '19915ef17940c15826c9aa86'
 'ec7849a2054a657faff6c708' '6ea44aa1a30436892616d661'
 'ce6dfb96987b441910429713' 'dae06a291871150de3b21aa2'
 '89bb0920e3606aaa96969594' '041d0707ef5311be4d22cdbf'
 'be69de7e

In [ ]:
# calculate completion time in the dataset

nc_df["disposition_date"] = pd.to_datetime(nc_df['disposition_date'])
nc_df["sentence_time"] = pd.to_timedelta(nc_df["sentence_time"], unit='D')
nc_df["adjusted_disposition_date"] = nc_df["disposition_date"] + \
    nc_df['sentence_time']

current_day = pd.to_datetime("06/01/2020", infer_datetime_format=True)
five_year_delta = pd.Timedelta("1826 days")
ten_year_delta = pd.Timedelta("3652 days")

nc_df["over_five_years"] = np.where((current_day - nc_df["adjusted_disposition_date"] ) > five_year_delta, "True", "False")
nc_df["over_ten_years"] = np.where((current_day - nc_df["adjusted_disposition_date"] ) > ten_year_delta, "True", "False")



#for index,row in nc_df.iterrows():
#    row["over_five_years"] = 'false'
#    row["over_ten_years"] = 'false'
#    if (current_day - row["adjusted_disposition_date"]) > five_year_delta:
#        row["over_five_years"] = 'true'
#    if (current_day - row["adjusted_disposition_date"]) > ten_year_delta:
#        row["over_ten_years"] = 'true'

#nc_df.to_csv("test.csv")

#nc_df["over_five_years"] = str((current_day - nc_df["adjusted_disposition_date"]) > five_year_delta)
#nc_df["over_ten_years"] = str((current_day - nc_df["adjusted_disposition_date"]) > ten_year_delta)

print(nc_df.iloc[:25])



                          id  year_birth  zipcode screening_created_at  scope  \
0   c53a005b0ba339fdd1870787        1980  27858.0            3/21/2017    7.0   
1   c53a005b0ba339fdd1870787        1980  27858.0            3/21/2017    7.0   
2   c53a005b0ba339fdd1870787        1980  27858.0            3/21/2017    7.0   
3   c53a005b0ba339fdd1870787        1980  27858.0            3/21/2017    7.0   
4   c53a005b0ba339fdd1870787        1980  27858.0            3/21/2017    7.0   
5   c53a005b0ba339fdd1870787        1980  27858.0            3/21/2017    7.0   
6   f38b52f8a17d908dc50dab90        1986  27858.0             5/7/2018    7.0   
7   f38b52f8a17d908dc50dab90        1986  27858.0             5/7/2018    7.0   
8   f38b52f8a17d908dc50dab90        1986  27858.0             5/7/2018    7.0   
9   f38b52f8a17d908dc50dab90        1986  27858.0             5/7/2018    7.0   
10  f38b52f8a17d908dc50dab90        1986  27858.0             5/7/2018    7.0   
11  f38b52f8a17d908dc50dab90

In [ ]:
# for each charge in the dataset:
#.  if charge is a conviction ("guilty" in disposition_normalized)
#     if it is a misdemeanor:
#       check if it is not an A1 misdemeanor (using Joe's brute force sheet)
#       if it is not, check the time since completion time
#       if more than 5 years, eligible

#   if it is a felony:
#     check if it is not an A-G felony
#.    if not, check time since completion

nc_df["Eligibility"] ='Not Eligible'
with open("eligibility_included.csv", "w+") as file:
    csv_writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(list(nc_df.columns.values))
    for index, row in nc_df.iterrows():
        id = row["id"]
        guilty = row["Disposition_Normalization"]
        if id in pending_ids:
            continue
        # if charge is a conviction
        elif guilty == "guilty":
            level = row["ChargeType_Normalization"]
            description = row["charge"]
            if level == "felony":
                if description in brute_force_dict:
                    if brute_force_dict[description] == "yes":
                        if row["over_ten_years"] == "True":
                            time = row["adjusted_disposition_date"]
                            freetime = time + pd.offsets.DateOffset(days=(365 * 10))
                            for item in final_dict[str(id)]:
                                print(item, time, freetime)
                                if item > time and item < freetime:
                                    print("inelgible because of concurrency")
                                    break
                            else:
                                row["Eligibility"] = "Eligible"
            elif level == "misdemeanor":
                if description in brute_force_dict:
                    if brute_force_dict[description] == "yes":
                        if row["over_five_years"] == "True":
                            time = row["adjusted_disposition_date"]
                            freetime = time + pd.offsets.DateOffset(days=(365 * 5))
                            for item in final_dict[str(id)]:
                                print(item, time, freetime)
                                if item > time and item < freetime:
                                    print("inelgible because of concurrency")
                                    break
                            else:
                                row["Eligibility"] = "Eligible"
        # if charge is not a conviction
        elif guilty == "not_guilty/dismissed":
            if row["id"] not in guilty_felony_ids:
                row["Eligibility"] = 'Eligible'

        csv_writer.writerow(row)


2009-05-28 00:00:00 2009-05-28 00:00:00 2014-05-27 00:00:00
1994-09-27 00:00:00 2009-05-28 00:00:00 2014-05-27 00:00:00
1994-09-27 00:00:00 2009-05-28 00:00:00 2014-05-27 00:00:00
2007-09-28 00:00:00 2008-09-22 00:00:00 2013-09-21 00:00:00
2007-09-28 00:00:00 2008-09-22 00:00:00 2013-09-21 00:00:00
2007-09-28 00:00:00 2008-09-22 00:00:00 2013-09-21 00:00:00
2007-09-28 00:00:00 2008-09-22 00:00:00 2013-09-21 00:00:00
2007-09-28 00:00:00 2008-09-22 00:00:00 2013-09-21 00:00:00
2007-09-28 00:00:00 2008-09-22 00:00:00 2013-09-21 00:00:00
1996-03-11 00:00:00 2008-09-22 00:00:00 2013-09-21 00:00:00
1996-03-11 00:00:00 2008-09-22 00:00:00 2013-09-21 00:00:00
1996-12-20 00:00:00 2008-09-22 00:00:00 2013-09-21 00:00:00
2007-09-28 00:00:00 2008-09-22 00:00:00 2013-09-21 00:00:00
1990-08-16 00:00:00 2012-02-03 00:00:00 2017-02-01 00:00:00
2011-02-08 00:00:00 2012-02-03 00:00:00 2017-02-01 00:00:00
1991-10-02 00:00:00 2012-02-03 00:00:00 2017-02-01 00:00:00
1991-10-02 00:00:00 2012-02-03 00:00:00 

2003-03-19 00:00:00 2011-02-25 00:00:00 2016-02-24 00:00:00
2006-03-23 00:00:00 2011-02-25 00:00:00 2016-02-24 00:00:00
2006-03-23 00:00:00 2011-02-25 00:00:00 2016-02-24 00:00:00
2006-03-23 00:00:00 2011-02-25 00:00:00 2016-02-24 00:00:00
2006-03-23 00:00:00 2011-02-25 00:00:00 2016-02-24 00:00:00
2006-03-23 00:00:00 2011-02-25 00:00:00 2016-02-24 00:00:00
2006-03-23 00:00:00 2011-02-25 00:00:00 2016-02-24 00:00:00
2006-03-23 00:00:00 2011-02-25 00:00:00 2016-02-24 00:00:00
2003-03-19 00:00:00 2011-02-25 00:00:00 2016-02-24 00:00:00
2006-03-23 00:00:00 2011-02-25 00:00:00 2016-02-24 00:00:00
2006-03-23 00:00:00 2011-02-25 00:00:00 2016-02-24 00:00:00
2006-03-23 00:00:00 2011-02-25 00:00:00 2016-02-24 00:00:00
2006-03-23 00:00:00 2011-02-25 00:00:00 2016-02-24 00:00:00
2006-03-23 00:00:00 2011-02-25 00:00:00 2016-02-24 00:00:00
2006-03-23 00:00:00 2011-02-25 00:00:00 2016-02-24 00:00:00
2006-03-23 00:00:00 2011-02-25 00:00:00 2016-02-24 00:00:00
2003-03-19 00:00:00 2006-03-23 00:00:00 

2001-07-25 00:00:00 2001-07-25 00:00:00 2006-07-24 00:00:00
2013-10-03 00:00:00 2013-10-03 00:00:00 2018-10-02 00:00:00
2013-11-12 00:00:00 2013-10-03 00:00:00 2018-10-02 00:00:00
inelgible because of concurrency
2013-10-03 00:00:00 2013-11-12 00:00:00 2018-11-11 00:00:00
2013-11-12 00:00:00 2013-11-12 00:00:00 2018-11-11 00:00:00
2018-04-04 00:00:00 2013-11-12 00:00:00 2018-11-11 00:00:00
inelgible because of concurrency
2011-08-26 00:00:00 2012-08-20 00:00:00 2017-08-19 00:00:00
2011-08-26 00:00:00 2012-08-20 00:00:00 2017-08-19 00:00:00
2011-08-26 00:00:00 2012-08-20 00:00:00 2017-08-19 00:00:00
2011-08-26 00:00:00 2012-08-20 00:00:00 2017-08-19 00:00:00
1992-05-26 00:00:00 1993-10-25 00:00:00 1998-10-24 00:00:00
1993-10-25 00:00:00 1993-10-25 00:00:00 1998-10-24 00:00:00
1993-10-25 00:00:00 1993-10-25 00:00:00 1998-10-24 00:00:00
1993-11-02 00:00:00 1993-10-25 00:00:00 1998-10-24 00:00:00
inelgible because of concurrency
1992-05-26 00:00:00 1993-10-25 00:00:00 1998-10-24 00:00:00
1

1992-08-11 00:00:00 1997-08-16 00:00:00 2002-08-15 00:00:00
1994-03-08 00:00:00 1997-08-16 00:00:00 2002-08-15 00:00:00
1994-03-08 00:00:00 1997-08-16 00:00:00 2002-08-15 00:00:00
1996-06-10 00:00:00 1997-08-16 00:00:00 2002-08-15 00:00:00
1996-05-28 00:00:00 1997-08-16 00:00:00 2002-08-15 00:00:00
1996-08-21 00:00:00 1997-08-16 00:00:00 2002-08-15 00:00:00
2000-03-21 00:00:00 1997-08-16 00:00:00 2002-08-15 00:00:00
inelgible because of concurrency
1986-02-25 00:00:00 2002-02-04 00:00:00 2007-02-03 00:00:00
1992-08-11 00:00:00 2002-02-04 00:00:00 2007-02-03 00:00:00
1994-03-08 00:00:00 2002-02-04 00:00:00 2007-02-03 00:00:00
1994-03-08 00:00:00 2002-02-04 00:00:00 2007-02-03 00:00:00
1996-06-10 00:00:00 2002-02-04 00:00:00 2007-02-03 00:00:00
1996-05-28 00:00:00 2002-02-04 00:00:00 2007-02-03 00:00:00
1996-08-21 00:00:00 2002-02-04 00:00:00 2007-02-03 00:00:00
2000-03-21 00:00:00 2002-02-04 00:00:00 2007-02-03 00:00:00
2002-02-04 00:00:00 2002-02-04 00:00:00 2007-02-03 00:00:00
2002-02

1993-11-09 00:00:00 1999-03-02 00:00:00 2009-02-27 00:00:00
1993-11-09 00:00:00 1999-03-02 00:00:00 2009-02-27 00:00:00
1993-11-09 00:00:00 1999-03-02 00:00:00 2009-02-27 00:00:00
1993-11-09 00:00:00 1999-03-02 00:00:00 2009-02-27 00:00:00
1993-11-09 00:00:00 1999-03-02 00:00:00 2009-02-27 00:00:00
1994-03-03 00:00:00 1999-03-02 00:00:00 2009-02-27 00:00:00
1994-03-03 00:00:00 1999-03-02 00:00:00 2009-02-27 00:00:00
1994-03-03 00:00:00 1999-03-02 00:00:00 2009-02-27 00:00:00
1994-03-03 00:00:00 1999-03-02 00:00:00 2009-02-27 00:00:00
1994-03-03 00:00:00 1999-03-02 00:00:00 2009-02-27 00:00:00
1993-11-09 00:00:00 1994-03-03 00:00:00 2004-02-29 00:00:00
1993-11-09 00:00:00 1994-03-03 00:00:00 2004-02-29 00:00:00
1993-11-09 00:00:00 1994-03-03 00:00:00 2004-02-29 00:00:00
1993-11-09 00:00:00 1994-03-03 00:00:00 2004-02-29 00:00:00
1993-11-09 00:00:00 1994-03-03 00:00:00 2004-02-29 00:00:00
1994-03-03 00:00:00 1994-03-03 00:00:00 2004-02-29 00:00:00
1994-03-03 00:00:00 1994-03-03 00:00:00 

...

2002-11-22 00:00:00 1994-04-12 00:00:00 1999-04-11 00:00:00
2002-11-22 00:00:00 1994-04-12 00:00:00 1999-04-11 00:00:00
1994-04-12 00:00:00 1994-04-12 00:00:00 1999-04-11 00:00:00
1994-05-16 00:00:00 1994-04-12 00:00:00 1999-04-11 00:00:00
inelgible because of concurrency
2002-11-22 00:00:00 1994-05-16 00:00:00 1999-05-15 00:00:00
2002-11-22 00:00:00 1994-05-16 00:00:00 1999-05-15 00:00:00
1994-04-12 00:00:00 1994-05-16 00:00:00 1999-05-15 00:00:00
1994-05-16 00:00:00 1994-05-16 00:00:00 1999-05-15 00:00:00
1994-05-16 00:00:00 1994-05-16 00:00:00 1999-05-15 00:00:00
1994-05-16 00:00:00 1994-05-16 00:00:00 1999-05-15 00:00:00
1995-10-23 00:00:00 1994-05-16 00:00:00 1999-05-15 00:00:00
inelgible because of concurrency
2002-11-22 00:00:00 1996-09-12 00:00:00 2001-09-11 00:00:00
2002-11-22 00:00:00 1996-09-12 00:00:00 2001-09-11 00:00:00
1994-04-12 00:00:00 1996-09-12 00:00:00 2001-09-11 00:00:00
1994-05-16 00:00:00 1996-09-12 00:00:00 2001-09-11 00:00:00
1994-05-16 00:00:00 1996-09-12 00:

2016-11-01 00:00:00 2009-01-30 00:00:00 2014-01-29 00:00:00
2016-11-01 00:00:00 2009-01-30 00:00:00 2014-01-29 00:00:00
2016-11-01 00:00:00 2009-01-30 00:00:00 2014-01-29 00:00:00
2016-11-01 00:00:00 2009-01-30 00:00:00 2014-01-29 00:00:00
1992-08-05 00:00:00 2009-01-30 00:00:00 2014-01-29 00:00:00
1993-01-27 00:00:00 2009-01-30 00:00:00 2014-01-29 00:00:00
1993-01-27 00:00:00 2009-01-30 00:00:00 2014-01-29 00:00:00
2003-10-23 00:00:00 2009-01-30 00:00:00 2014-01-29 00:00:00
2003-10-23 00:00:00 2009-01-30 00:00:00 2014-01-29 00:00:00
2012-07-03 00:00:00 2009-01-30 00:00:00 2014-01-29 00:00:00
inelgible because of concurrency
1993-03-01 00:00:00 1993-03-01 00:00:00 1998-02-28 00:00:00
1993-03-01 00:00:00 1993-03-01 00:00:00 1998-02-28 00:00:00
1993-03-01 00:00:00 1993-03-01 00:00:00 1998-02-28 00:00:00
1993-03-01 00:00:00 1993-03-01 00:00:00 1998-02-28 00:00:00
1993-03-01 00:00:00 1993-03-01 00:00:00 1998-02-28 00:00:00
1993-03-01 00:00:00 1993-03-01 00:00:00 1998-02-28 00:00:00
1993-03

1993-01-11 00:00:00 1994-09-07 00:00:00 2004-09-04 00:00:00
1994-09-07 00:00:00 1994-09-07 00:00:00 2004-09-04 00:00:00
1999-01-14 00:00:00 1994-09-07 00:00:00 2004-09-04 00:00:00
inelgible because of concurrency
2009-07-06 00:00:00 2006-03-07 00:00:00 2011-03-06 00:00:00
inelgible because of concurrency
2009-07-06 00:00:00 2011-12-02 00:00:00 2016-11-30 00:00:00
2009-07-06 00:00:00 2011-12-02 00:00:00 2016-11-30 00:00:00
2009-07-06 00:00:00 2011-12-02 00:00:00 2016-11-30 00:00:00
2003-10-16 00:00:00 2011-12-02 00:00:00 2016-11-30 00:00:00
2005-01-26 00:00:00 2011-12-02 00:00:00 2016-11-30 00:00:00
2005-01-26 00:00:00 2011-12-02 00:00:00 2016-11-30 00:00:00
2007-01-10 00:00:00 2011-12-02 00:00:00 2016-11-30 00:00:00
2008-08-06 00:00:00 2011-12-02 00:00:00 2016-11-30 00:00:00
2010-02-10 00:00:00 2011-12-02 00:00:00 2016-11-30 00:00:00
2010-02-10 00:00:00 2011-12-02 00:00:00 2016-11-30 00:00:00
2011-04-27 00:00:00 2011-12-02 00:00:00 2016-11-30 00:00:00
1997-01-14 00:00:00 1986-01-09 00:

1997-01-14 00:00:00 1988-09-14 00:00:00 1993-09-13 00:00:00
1986-01-09 00:00:00 1988-09-14 00:00:00 1993-09-13 00:00:00
1986-01-09 00:00:00 1988-09-14 00:00:00 1993-09-13 00:00:00
1986-01-09 00:00:00 1988-09-14 00:00:00 1993-09-13 00:00:00
1990-07-17 00:00:00 1988-09-14 00:00:00 1993-09-13 00:00:00
inelgible because of concurrency
2010-06-29 00:00:00 2015-02-02 00:00:00 2020-02-01 00:00:00
2010-09-21 00:00:00 2015-02-02 00:00:00 2020-02-01 00:00:00
2010-09-21 00:00:00 2015-02-02 00:00:00 2020-02-01 00:00:00
2010-09-21 00:00:00 2015-02-02 00:00:00 2020-02-01 00:00:00
2012-02-28 00:00:00 2015-02-02 00:00:00 2020-02-01 00:00:00
2012-02-28 00:00:00 2015-02-02 00:00:00 2020-02-01 00:00:00
2012-02-28 00:00:00 2015-02-02 00:00:00 2020-02-01 00:00:00
2012-02-28 00:00:00 2015-02-02 00:00:00 2020-02-01 00:00:00
2013-12-12 00:00:00 2015-02-02 00:00:00 2020-02-01 00:00:00
2016-08-12 00:00:00 2015-02-02 00:00:00 2020-02-01 00:00:00
inelgible because of concurrency
2004-03-11 00:00:00 2006-03-31 00:

2012-06-01 00:00:00 2012-03-21 00:00:00 2017-03-20 00:00:00
inelgible because of concurrency
2001-11-19 00:00:00 2002-02-13 00:00:00 2007-02-12 00:00:00
2001-11-19 00:00:00 2002-02-13 00:00:00 2007-02-12 00:00:00
2001-11-19 00:00:00 2002-02-13 00:00:00 2007-02-12 00:00:00
2001-11-19 00:00:00 2002-02-13 00:00:00 2007-02-12 00:00:00
2001-09-10 00:00:00 2002-02-13 00:00:00 2007-02-12 00:00:00
2002-01-07 00:00:00 2002-02-13 00:00:00 2007-02-12 00:00:00
2002-01-31 00:00:00 2002-02-13 00:00:00 2007-02-12 00:00:00
2002-03-13 00:00:00 2002-02-13 00:00:00 2007-02-12 00:00:00
inelgible because of concurrency
2003-11-17 00:00:00 2013-05-21 00:00:00 2018-05-20 00:00:00
2007-04-03 00:00:00 2013-05-21 00:00:00 2018-05-20 00:00:00
2010-10-18 00:00:00 2013-05-21 00:00:00 2018-05-20 00:00:00
2012-05-24 00:00:00 2013-05-21 00:00:00 2018-05-20 00:00:00
2012-06-15 00:00:00 2013-05-21 00:00:00 2018-05-20 00:00:00
2003-11-17 00:00:00 2012-06-15 00:00:00 2017-06-14 00:00:00
2007-04-03 00:00:00 2012-06-15 00:

## Counting

In [ ]:
# Number of people in the sample
print("Number of charges in sample: " + str(len(nc_df.index)))
print("Number of people in sample: " + str(nc_df.id.nunique()))

conviction_df = nc_df[nc_df["Disposition_Normalization"].str.match("guilty",na=False)]
print("Number of people with convictions: " + str(conviction_df.id.nunique()))

felony_conviction_df = conviction_df[conviction_df["ChargeType_Normalization"].str.contains("felony",na=False)]
print('number of people with felony convictions: ' + str(felony_conviction_df.id.nunique()))

misdemeanor_conviction_df = conviction_df[conviction_df["ChargeType_Normalization"].str.contains("misdemeanor",na=False)]
print('number of people with misdemeanor convictions: ' + str(misdemeanor_conviction_df.id.nunique()))

felony_df = nc_df[nc_df["ChargeType_Normalization"].str.contains("felony",na=False)]
print('number of people with felony charges: ' + str(felony_df.id.nunique()))

Number of charges in sample: 31572
Number of people in sample: 2492
Number of people with convictions: 1851
number of people with felony convictions: 263
number of people with misdemeanor convictions: 1156
number of people with felony charges: 559


In [ ]:
print(nc_df["ChargeType_Normalization"].unique())

['felony' 'misdemeanor' 'minor' 'not classified']


In [ ]:
ei_df = pd.read_csv("eligibility_included.csv")

has_eligible_df = ei_df[ei_df["Eligibility"].str.match("Eligible",na=False)]
print("Number of people with records eligible: " + str(has_eligible_df.id.nunique()))

has_eligible_felony_df = has_eligible_df[has_eligible_df["ChargeType_Normalization"].str.match("felony",na=False)]
print("Number of people with felony records eligible: " + str(has_eligible_felony_df.id.nunique()))

has_eligible_misdemeanor_df = has_eligible_df[has_eligible_df["ChargeType_Normalization"].str.match("misdemeanor",na=False)]
print("Number of people with misdemeanor records eligible: " + str(has_eligible_misdemeanor_df.id.nunique()))

new_convictions_df = ei_df[ei_df["Disposition_Normalization"].str.match("guilty",na=False)]


Number of people with records eligible: 1495
Number of people with felony records eligible: 18
Number of people with misdemeanor records eligible: 1101


In [ ]:
# of people who can clear all convictions
total = 0
new_convictions_df = ei_df[ei_df["Disposition_Normalization"].str.match("guilty",na=False)]
id_list = nc_df.id.unique()
for person_id in id_list:
    temp_df = new_convictions_df[new_convictions_df["id"].str.match(person_id, na=False)]
    eligibility_values = temp_df["Eligibility"].unique()
    if "Not Eligible" not in eligibility_values and len(eligibility_values) == 1:
        total += 1
print("Number of people who can clear all convictions: " + str(total))

Number of people who can clear all convictions: 164


In [ ]:
id_list = nc_df.id.unique()
fully_eligible = 0
partially_eligible = 0
none_eligible = 0
for person_id in id_list:
    temp_df = ei_df[ei_df["id"].str.match(person_id, na=False)]
    temp_eligible_df = temp_df[temp_df["Eligibility"].str.match("Eligible", na=False)]
    temp_not_eligible_df = temp_df[temp_df["Eligibility"].str.match("Not Eligible", na=False)]

    if len(temp_not_eligible_df) > 0 and len(temp_eligible_df) > 0:
        partially_eligible += 1
    elif len(temp_eligible_df) == 0:
        none_eligible += 1
    elif len(temp_not_eligible_df) == 0:
        temp_eligible_felonies_df = temp_eligible_df[temp_eligible_df["ChargeType_Normalization"].str.match("felony", na=False)]
        if len(temp_eligible_felonies_df) > 0:
            if temp_eligible_felonies_df.disposition_date.nunique() > 1:
                partially_eligible += 1
            else:
                fully_eligible += 1
        else:
            fully_eligible += 1
    del(temp_df)
    del(temp_eligible_df)
    del(temp_not_eligible_df)

print("Fully eligible: " + str(fully_eligible))
print('Partially eligible: ' + str(partially_eligible))
print('None eligible: ' + str(none_eligible))
print('Total: ' + str(fully_eligible + partially_eligible + none_eligible))

Fully eligible: 532
Partially eligible: 963
None eligible: 997
Total: 2492


In [ ]:
# of people with only not guilty charges not guilty

only_non_guilties_list = []

id_list = nc_df.id.unique()
for id in id_list:
    temp_df = nc_df[nc_df["id"].str.match(id, na=False)]
    dispositions = temp_df["Disposition_Normalization"].unique()
    print(dispositions)
    if "not_guilty/dismissed" in dispositions and len(dispositions) == 1:
        only_non_guilties_list.append(id)
    del(temp_df)

print(only_non_guilties_list)
print(len(only_non_guilties_list))





['unknown' 'guilty' 'not_guilty/dismissed' 'likely_pending-blank']
['not_guilty/dismissed' 'unknown' 'pending' 'guilty']
['not_guilty/dismissed' 'guilty' 'unknown' 'pending']
['guilty' 'not_guilty/dismissed' 'unknown']
['not_guilty/dismissed' 'guilty']
['unknown' 'not_guilty/dismissed' 'guilty']
['guilty' 'not_guilty/dismissed' 'pending' 'unknown']
['guilty']
['pending' 'not_guilty/dismissed']
['guilty' 'not_guilty/dismissed' 'likely_pending-blank']
['guilty' 'likely_pending-blank']
['guilty' 'likely_pending-blank' 'not_guilty/dismissed']
['not_guilty/dismissed' 'guilty' 'unknown']
['not_guilty/dismissed' 'guilty' 'unknown']
['guilty']
['guilty' 'not_guilty/dismissed' 'unknown' 'likely_pending-blank']
['likely_pending-blank' 'pending' 'guilty' 'unknown'
 'not_guilty/dismissed']
['guilty' 'unknown']
['not_guilty/dismissed' 'guilty' 'likely_pending-blank']
['not_guilty/dismissed' 'guilty' 'unknown']
['guilty' 'unknown' 'not_guilty/dismissed']
['not_guilty/dismissed' 'unknown' 'guilty' 'p

['unknown' 'not_guilty/dismissed']
['guilty' 'not_guilty/dismissed']
['not_guilty/dismissed' 'unknown']
['not_guilty/dismissed']
['likely_pending-blank']
['not_guilty/dismissed']
['guilty' 'not_guilty/dismissed' 'pending']
['guilty' 'not_guilty/dismissed' 'unknown' 'pending']
['unknown' 'not_guilty/dismissed' 'pending' 'guilty']
['unknown' 'not_guilty/dismissed']
['guilty']
['not_guilty/dismissed']
['guilty' 'not_guilty/dismissed' 'unknown' 'pending']
['not_guilty/dismissed' 'guilty' 'unknown']
['guilty' 'not_guilty/dismissed']
['not_guilty/dismissed' 'guilty']
['guilty']
['not_guilty/dismissed']
['pending']
['not_guilty/dismissed']
['guilty' 'not_guilty/dismissed']
['not_guilty/dismissed']
['unknown' 'not_guilty/dismissed']
['guilty' 'not_guilty/dismissed']
['not_guilty/dismissed' 'guilty']
['not_guilty/dismissed' 'unknown']
['guilty']
['guilty' 'not_guilty/dismissed']
['not_guilty/dismissed' 'pending' 'guilty']
['pending' 'guilty' 'not_guilty/dismissed']
['not_guilty/dismissed' 'guil

['likely_pending-blank' 'guilty' 'not_guilty/dismissed' 'unknown']
['likely_pending-blank' 'guilty' 'not_guilty/dismissed' 'unknown']
['likely_pending-blank' 'guilty' 'unknown' 'not_guilty/dismissed']
['likely_pending-blank' 'not_guilty/dismissed' 'guilty']
['likely_pending-blank' 'not_guilty/dismissed' 'guilty' 'unknown']
['likely_pending-blank' 'guilty' 'not_guilty/dismissed' 'unknown']
['likely_pending-blank' 'unknown' 'not_guilty/dismissed' 'guilty']
['likely_pending-blank' 'guilty' 'unknown' 'not_guilty/dismissed']
['likely_pending-blank' 'not_guilty/dismissed' 'unknown' 'guilty']
['likely_pending-blank' 'guilty' 'pending' 'not_guilty/dismissed']
['likely_pending-blank' 'not_guilty/dismissed' 'guilty' 'unknown']
['likely_pending-blank' 'not_guilty/dismissed' 'guilty' 'unknown']
['likely_pending-blank' 'guilty' 'not_guilty/dismissed' 'unknown'
 'pending']
['likely_pending-blank' 'unknown' 'guilty' 'not_guilty/dismissed']
['likely_pending-blank' 'not_guilty/dismissed' 'guilty']
['li

['not_guilty/dismissed']
['guilty' 'not_guilty/dismissed']
['not_guilty/dismissed' 'unknown' 'pending']
['not_guilty/dismissed' 'guilty']
['guilty']
['not_guilty/dismissed' 'guilty' 'unknown']
['not_guilty/dismissed']
['guilty' 'not_guilty/dismissed']
['guilty' 'not_guilty/dismissed']
['guilty' 'unknown' 'not_guilty/dismissed']
['not_guilty/dismissed' 'guilty' 'pending' 'likely_pending-blank']
['not_guilty/dismissed' 'guilty']
['guilty' 'not_guilty/dismissed']
['not_guilty/dismissed' 'unknown' 'guilty']
['not_guilty/dismissed' 'guilty' 'pending' 'unknown']
['guilty' 'unknown' 'not_guilty/dismissed' 'pending']
['guilty']
['not_guilty/dismissed' 'unknown']
['not_guilty/dismissed']
['not_guilty/dismissed' 'guilty']
['not_guilty/dismissed' 'guilty' 'pending']
['guilty' 'not_guilty/dismissed']
['not_guilty/dismissed' 'guilty']
['not_guilty/dismissed' 'guilty']
['not_guilty/dismissed' 'guilty']
['guilty']
['guilty' 'not_guilty/dismissed' 'unknown']
['not_guilty/dismissed' 'pending' 'unknown'

['not_guilty/dismissed' 'unknown']
['guilty' 'not_guilty/dismissed' 'unknown']
['not_guilty/dismissed' 'guilty' 'pending']
['not_guilty/dismissed']
['not_guilty/dismissed']
['guilty' 'unknown' 'not_guilty/dismissed']
['not_guilty/dismissed' 'guilty' 'unknown']
['not_guilty/dismissed' 'unknown']
['unknown' 'guilty']
['pending' 'not_guilty/dismissed']
['guilty' 'not_guilty/dismissed' 'unknown']
['unknown' 'not_guilty/dismissed']
['guilty' 'not_guilty/dismissed' 'pending']
['unknown' 'guilty' 'not_guilty/dismissed']
['not_guilty/dismissed']
['guilty' 'not_guilty/dismissed' 'pending' 'unknown']
['pending']
['not_guilty/dismissed' 'guilty']
['guilty' 'not_guilty/dismissed']
['not_guilty/dismissed' 'unknown' 'guilty']
['unknown' 'not_guilty/dismissed' 'guilty']
['guilty']
['guilty' 'not_guilty/dismissed' 'unknown']
['not_guilty/dismissed']
['not_guilty/dismissed' 'unknown' 'guilty']
['not_guilty/dismissed' 'pending']
['guilty' 'not_guilty/dismissed']
['guilty' 'not_guilty/dismissed']
['not_g

['unknown']
['not_guilty/dismissed']
['guilty' 'not_guilty/dismissed' 'unknown']
['guilty' 'not_guilty/dismissed' 'unknown']
['not_guilty/dismissed' 'guilty']
['not_guilty/dismissed']
['not_guilty/dismissed' 'unknown' 'guilty']
['guilty']
['guilty' 'not_guilty/dismissed']
['guilty' 'not_guilty/dismissed' 'unknown']
['guilty' 'unknown']
['guilty' 'not_guilty/dismissed']
['guilty' 'not_guilty/dismissed' 'unknown']
['not_guilty/dismissed']
['guilty' 'not_guilty/dismissed' 'unknown' 'pending']
['not_guilty/dismissed' 'guilty']
['not_guilty/dismissed' 'guilty']
['guilty' 'not_guilty/dismissed']
['not_guilty/dismissed' 'guilty' 'unknown']
['guilty' 'not_guilty/dismissed' 'unknown']
['guilty']
['unknown' 'not_guilty/dismissed' 'guilty' 'pending']
['not_guilty/dismissed' 'unknown' 'guilty']
['guilty']
['guilty' 'not_guilty/dismissed' 'unknown']
['guilty']
['not_guilty/dismissed' 'guilty' 'unknown']
['unknown']
['not_guilty/dismissed' 'guilty']
['guilty']
['guilty' 'not_guilty/dismissed']
['gui

['unknown' 'guilty']
['not_guilty/dismissed']
['not_guilty/dismissed' 'guilty']
['not_guilty/dismissed']
['not_guilty/dismissed' 'guilty' 'unknown']
['guilty']
['guilty' 'not_guilty/dismissed']
['guilty' 'not_guilty/dismissed' 'unknown']
['pending']
['guilty']
['not_guilty/dismissed']
['guilty' 'unknown']
['not_guilty/dismissed' 'unknown']
['guilty']
['guilty' 'not_guilty/dismissed' 'pending']
['not_guilty/dismissed' 'guilty' 'unknown']
['guilty' 'unknown' 'pending']
['unknown']
['guilty']
['not_guilty/dismissed']
['not_guilty/dismissed' 'unknown']
['guilty' 'not_guilty/dismissed' 'unknown' 'pending']
['not_guilty/dismissed' 'guilty']
['guilty']
['guilty' 'not_guilty/dismissed' 'unknown']
['not_guilty/dismissed' 'guilty' 'unknown']
['guilty']
['guilty' 'pending' 'not_guilty/dismissed']
['guilty']
['unknown' 'guilty' 'not_guilty/dismissed']
['unknown']
['pending']
['not_guilty/dismissed' 'guilty']
['guilty' 'not_guilty/dismissed']
['guilty' 'not_guilty/dismissed']
['guilty' 'not_guilty/

['guilty']
['not_guilty/dismissed']
['guilty']
['guilty' 'not_guilty/dismissed' 'unknown']
['not_guilty/dismissed' 'guilty']
['guilty']
['not_guilty/dismissed' 'guilty']
['guilty']
['guilty']
['guilty' 'not_guilty/dismissed']
['guilty']
['guilty' 'not_guilty/dismissed']
['not_guilty/dismissed' 'unknown']
['guilty' 'not_guilty/dismissed' 'unknown']
['guilty' 'not_guilty/dismissed' 'unknown']
['not_guilty/dismissed']
['guilty' 'not_guilty/dismissed']
['guilty' 'not_guilty/dismissed' 'pending']
['guilty' 'not_guilty/dismissed' 'unknown']
['not_guilty/dismissed' 'guilty']
['guilty' 'not_guilty/dismissed' 'pending']
['not_guilty/dismissed' 'guilty' 'unknown']
['guilty' 'not_guilty/dismissed' 'unknown']
['pending' 'unknown']
['pending' 'not_guilty/dismissed']
['unknown']
['guilty' 'not_guilty/dismissed']
['guilty' 'not_guilty/dismissed' 'unknown']
['guilty' 'not_guilty/dismissed']
['not_guilty/dismissed' 'guilty' 'unknown']
['guilty' 'not_guilty/dismissed']
['guilty' 'not_guilty/dismissed' '

['not_guilty/dismissed' 'unknown' 'guilty']
['not_guilty/dismissed' 'guilty' 'unknown']
['guilty']
['guilty']
['not_guilty/dismissed' 'guilty']
['not_guilty/dismissed']
['guilty']
['not_guilty/dismissed']
['guilty']
['guilty']
['not_guilty/dismissed' 'guilty']
['not_guilty/dismissed' 'guilty']
['guilty']
['not_guilty/dismissed']
['not_guilty/dismissed']
['not_guilty/dismissed' 'guilty']
['guilty' 'not_guilty/dismissed']
['not_guilty/dismissed']
['unknown' 'guilty']
['guilty' 'not_guilty/dismissed' 'unknown']
['guilty']
['guilty']
['guilty' 'not_guilty/dismissed']
['guilty' 'not_guilty/dismissed']
['unknown' 'guilty']
['unknown' 'not_guilty/dismissed']
['guilty']
['guilty']
['not_guilty/dismissed']
['not_guilty/dismissed']
['guilty']
['guilty']
['guilty' 'not_guilty/dismissed' 'unknown']
['not_guilty/dismissed' 'pending']
['not_guilty/dismissed']
['guilty']
['unknown' 'not_guilty/dismissed']
['pending' 'unknown']
['not_guilty/dismissed' 'guilty']
['guilty' 'not_guilty/dismissed']
['not_

['guilty' 'not_guilty/dismissed']
['not_guilty/dismissed']
['guilty']
['not_guilty/dismissed']
['not_guilty/dismissed']
['guilty' 'not_guilty/dismissed']
['not_guilty/dismissed']
['guilty']
['not_guilty/dismissed']
['guilty']
['guilty' 'unknown' 'not_guilty/dismissed']
['guilty' 'not_guilty/dismissed' 'unknown']
['not_guilty/dismissed' 'guilty']
['unknown']
['unknown']
['not_guilty/dismissed' 'guilty' 'unknown']
['not_guilty/dismissed' 'guilty']
['unknown' 'guilty' 'not_guilty/dismissed']
['not_guilty/dismissed' 'guilty']
['not_guilty/dismissed']
['not_guilty/dismissed']
['guilty']
['guilty']
['guilty']
['guilty']
['guilty']
['guilty']
['guilty']
['guilty']
['guilty']
['guilty']
['guilty']
['guilty']
['guilty' 'unknown' 'not_guilty/dismissed']
['unknown' 'guilty']
['not_guilty/dismissed']
['pending' 'guilty' 'unknown' 'not_guilty/dismissed']
['guilty' 'not_guilty/dismissed' 'unknown']
['not_guilty/dismissed']
['guilty']
['guilty']
['guilty']
['not_guilty/dismissed']
['guilty' 'not_guil

In [ ]:
# of people with any felonies in teh sample

felony_sub_df = nc_df[nc_df['ChargeType_Normalization'].str.match("felony", na=False)]
print(felony_sub_df["id"].nunique())

559


In [ ]:
#top 10 charges

print(nc_df["charge"].value_counts())

UNAVAILABLE                                                                                          3268
EXPIRED REGISTRATION CARD/TAG (STATUTE: 20-111(2))                                                   1438
DWLR (STATUTE: 20-28(A))                                                                             1107
NO OPERATORS LICENSE (STATUTE: 20-7(A))                                                              1021
SPEEDING (STATUTE: 20-141(B))                                                                         906
SIMPLE WORTHLESS CHECK (STATUTE: 14-107(D)(1))                                                        722
EXPIRED/NO INSPECTION (STATUTE: 20-183.8(A)(1))                                                       704
OPERATE VEH NO INS (STATUTE: 20-313(A))                                                               692
IMPROPER EQUIP - SPEEDOMETER (STATUTE: 20-123.2)                                                      644
SPEEDING (STATUTE: 20-141(J1))                

In [ ]:
# and top 10 expungable charges
print(has_eligible_df["charge"].value_counts())

EXPIRED REGISTRATION CARD/TAG (STATUTE: 20-111(2))                          693
EXPIRED/NO INSPECTION (STATUTE: 20-183.8(A)(1))                             322
DWLR (STATUTE: 20-28(A))                                                    319
NO OPERATORS LICENSE (STATUTE: 20-7(A))                                     300
OPERATE VEH NO INS (STATUTE: 20-313(A))                                     292
SPEEDING (STATUTE: 20-141(J1))                                              188
MISDEMEANOR LARCENY (STATUTE: 14-72(A))                                     171
SIMPLE WORTHLESS CHECK (STATUTE: 14-107(D)(1))                              137
POSSESS DRUG PARAPHERNALIA (STATUTE: 90-113.22(A))                          127
EXPIRED REGISTRATION CARD/TAG                                               119
POSSESS MARIJUANA UP TO 1/2 OZ (STATUTE: 90-95(D)(4))                       115
FICT/ALT TITLE/REG CARD/TAG (STATUTE: 20-111(2))                            108
DWLR NOT IMPAIRED REV (STATUTE: 20-28(A)

In [ ]:
print(sorted(nc_df["disposition_date"].value_counts().index))

[Timestamp('1976-05-26 00:00:00'), Timestamp('1980-07-23 00:00:00'), Timestamp('1980-09-04 00:00:00'), Timestamp('1981-01-26 00:00:00'), Timestamp('1981-03-25 00:00:00'), Timestamp('1981-03-30 00:00:00'), Timestamp('1981-04-16 00:00:00'), Timestamp('1981-06-05 00:00:00'), Timestamp('1982-07-28 00:00:00'), Timestamp('1982-08-31 00:00:00'), Timestamp('1982-09-03 00:00:00'), Timestamp('1982-09-17 00:00:00'), Timestamp('1982-09-24 00:00:00'), Timestamp('1982-11-08 00:00:00'), Timestamp('1983-01-06 00:00:00'), Timestamp('1983-01-13 00:00:00'), Timestamp('1983-01-14 00:00:00'), Timestamp('1983-01-31 00:00:00'), Timestamp('1983-02-16 00:00:00'), Timestamp('1983-02-28 00:00:00'), Timestamp('1983-03-31 00:00:00'), Timestamp('1983-04-06 00:00:00'), Timestamp('1983-04-18 00:00:00'), Timestamp('1983-04-26 00:00:00'), Timestamp('1983-04-28 00:00:00'), Timestamp('1983-05-09 00:00:00'), Timestamp('1983-05-23 00:00:00'), Timestamp('1983-06-16 00:00:00'), Timestamp('1983-06-30 00:00:00'), Timestamp('19